## Compile & Clean Data

In [ ]:
import pandas as pd
# pd.read_parquet("/Users/scott.mckean/Desktop/3W/0/WELL-00001_20170201010207.parquet")
pd.read_parquet("3w_dataset_test/0/WELL-00001_20170201010207.parquet").reset_index(drop=False)

In [1]:
from hydrate.process import read_nested_parquet_files

In [ ]:
is_drawn = all_data.source_file.str.contains('DRAWN')
is_simulated = all_data.source_file.str.contains('SIMULATED')
real_data = all_data[~is_drawn & ~is_simulated]

In [ ]:
real_no_class = real_data['class'].isna()
real_no_timestamp = real_data['timestamp'].isna()
clean_data = real_data[~real_no_class & ~real_no_timestamp].copy()

In [ ]:
clean_data['well_number'] = clean_data['source_file'].str.extract(r'WELL-(\d+)').astype(int)

In [ ]:
clean_data['source_file'] = clean_data['source_file'].str.replace('/Users/scott.mckean/Desktop', '')

## Exploratory Data Analysis on Compiled Data

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_parquet('/Users/scott.mckean/Desktop/3w_compiled_data.parquet')
df.sample(10)

In [ ]:
events_names = {
    0: 'Normal',
    1: 'Abrupt Increase of BSW',
    2: 'Spurious Closure of DHSV',
    3: 'Severe Slugging',
    4: 'Flow Instability',
    5: 'Rapid Productivity Loss',
    6: 'Quick Restriction in PCK',
    7: 'Scaling in PCK',
    8: 'Hydrate in Production Line'
    }

tag_info = {
    1: {"tag": "P-PDG", "name": "Pressure at the PDG", "unit": "Pa"},
    2: {"tag": "P-TPT", "name": "Pressure at the TPT", "unit": "Pa"},
    3: {"tag": "T-TPT", "name": "Temperature at the TPT", "unit": "degC"},
    4: {"tag": "P-MON-CKP", "name": "Pressure upstream of the PCK", "unit": "Pa"},
    5: {"tag": "T-JUS-CKP", "name": "Temperature downstream of the PCK", "unit": "degC"},
    6: {"tag": "P-JUS-CKGL", "name": "Pressure downstream of the GLCK", "unit": "Pa"},
    7: {"tag": "T-JUS-CKGL", "name": "Temperature downstream of the GLCK", "unit": "degC"},
    8: {"tag": "QGL", "name": "Gas lift flow rate", "unit": "sm^3/s"}
}